In [5]:
import polars as pl

In [6]:
vtv_news = pl.read_ipc("data/vtv_articles_2.feather")

vtv_news = pl.concat([
    vtv_news,
    pl.read_ipc("data/vtv_articles.feather")
])

could not mmap compressed IPC file, defaulting to normal readcould not mmap compressed IPC file, defaulting to normal read

In [7]:
vtv_news.unique(subset="link")

link,title,datetime,description
str,str,str,str
"""https://vtv.vn...","""Dưới bóng cây ...","""10/03/2023 17:...","""VTV.vn - Son t..."
"""https://vtv.vn...","""Công nghệ AI c...","""10/03/2023 11:...","""VTV.vn - Tổ ch..."
"""https://vtv.vn...","""Ai là triệu ph...","""10/03/2023 10:...","""VTV.vn - Cậu b..."
"""https://vtv.vn...","""Meta quyết tâm...","""09/03/2023 15:...","""VTV.vn - Bất c..."
"""https://vtv.vn...","""Ai Cập: Tàu hỏ...","""08/03/2023 11:...","""VTV.vn - Ít nh..."
"""https://vtv.vn...","""Dưới bóng cây ...","""08/03/2023 11:...","""VTV.vn - Chạm ..."
"""https://vtv.vn...","""Ai sẽ trở thàn...","""06/03/2023 17:...","""VTV.vn - Trong..."
"""https://vtv.vn...","""Trung Quốc tăn...","""06/03/2023 06:...","""VTV.vn - Ngày ..."
"""https://vtv.vn...","""Mong Vua tiếng...","""05/03/2023 09:...","""VTV.vn - Theo ..."
